In [18]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [19]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [20]:
df_r_raw = pd.read_csv(url_cum)
df_r_raw

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,2020-11-09T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1303,187,1490,20449,...,472,1023,9652,703,19998.0,12296.0,32294,761552,529908.0,NaN
5456,2020-11-09T17:00:00,ITA,9,Toscana,43.769231,11.255889,1542,234,1776,42704,...,1033,2244,19116,1594,52685.0,12505.0,65190,1227725,806480.0,effettuati 26463 tamponi rapidi antigenici di...
5457,2020-11-09T17:00:00,ITA,10,Umbria,43.106758,12.388247,360,64,424,9572,...,196,314,5060,197,4303.0,10950.0,15253,330852,189991.0,NaN
5458,2020-11-09T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,159,15,174,2092,...,94,201,1863,207,3822.0,514.0,4336,45993,27671.0,NaN


In [21]:
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_r_raw.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,2020-11-09T17:00:00,19,Sicilia,1303,187,1490,20449,21939,472,1023,9652,703,19998.0,12296.0,32294,761552,529908.0
5456,2020-11-09T17:00:00,9,Toscana,1542,234,1776,42704,44480,1033,2244,19116,1594,52685.0,12505.0,65190,1227725,806480.0
5457,2020-11-09T17:00:00,10,Umbria,360,64,424,9572,9996,196,314,5060,197,4303.0,10950.0,15253,330852,189991.0
5458,2020-11-09T17:00:00,2,Valle d'Aosta,159,15,174,2092,2266,94,201,1863,207,3822.0,514.0,4336,45993,27671.0


In [22]:
# Current state S
df_raw_r_1 = df_rd[-21:].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-09T17:00:00,1,Piemonte,4540,312,4852,51404,56256,2059,2876,39721,4678,43124.0,57531.0,100655,1162525,735109.0
1,2020-11-09T17:00:00,2,Valle d'Aosta,159,15,174,2092,2266,94,201,1863,207,3822.0,514.0,4336,45993,27671.0
2,2020-11-09T17:00:00,3,Lombardia,6414,670,7084,124375,131459,-951,4777,115630,18442,203758.0,61773.0,265531,3277767,2014134.0
3,2020-11-09T17:00:00,5,Veneto,1464,195,1659,51503,53162,2192,2223,25735,2595,26203.0,55289.0,81492,2462529,953936.0
4,2020-11-09T17:00:00,6,Friuli Venezia Giulia,319,46,365,7467,7832,191,320,6760,456,13080.0,1968.0,15048,583139,243252.0
5,2020-11-09T17:00:00,7,Liguria,1387,92,1479,11242,12721,313,538,22310,1937,25834.0,11134.0,36968,489301,249942.0
6,2020-11-09T17:00:00,8,Emilia-Romagna,1939,204,2143,37487,39630,1778,2025,28978,4824,50134.0,23298.0,73432,1740948,922303.0
7,2020-11-09T17:00:00,9,Toscana,1542,234,1776,42704,44480,1033,2244,19116,1594,52685.0,12505.0,65190,1227725,806480.0
8,2020-11-09T17:00:00,10,Umbria,360,64,424,9572,9996,196,314,5060,197,4303.0,10950.0,15253,330852,189991.0
9,2020-11-09T17:00:00,11,Marche,519,71,590,9861,10451,210,262,7543,1057,18940.0,111.0,19051,346762,203665.0


In [23]:
# State S - 1
df_raw_r_0 = df_rd[-42:-21].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-08T17:00:00,1,Piemonte,4367,304,4671,49526,54197,2708,3884,38953,4629,42321.0,55458.0,97779,1149489,726199.0
1,2020-11-08T17:00:00,2,Valle d'Aosta,158,14,172,2000,2172,-9,55,1758,205,3634.0,501.0,4135,45323,27399.0
2,2020-11-08T17:00:00,3,Lombardia,6225,650,6875,125535,132410,4781,6318,110001,18343,201314.0,59440.0,260754,3256646,2003397.0
3,2020-11-08T17:00:00,5,Veneto,1355,185,1540,49430,50970,3313,3362,25725,2574,26082.0,53187.0,79269,2448503,948701.0
4,2020-11-08T17:00:00,6,Friuli Venezia Giulia,301,45,346,7295,7641,281,504,6636,451,12787.0,1941.0,14728,580815,242297.0
5,2020-11-08T17:00:00,7,Liguria,1334,81,1415,10993,12408,300,886,22106,1916,25326.0,11104.0,36430,486171,248706.0
6,2020-11-08T17:00:00,8,Emilia-Romagna,1836,194,2030,35822,37852,2203,2360,28740,4816,48938.0,22470.0,71408,1728477,914731.0
7,2020-11-08T17:00:00,9,Toscana,1474,226,1700,41747,43447,1864,2479,17938,1561,50905.0,12041.0,62946,1214035,799345.0
8,2020-11-08T17:00:00,10,Umbria,353,62,415,9385,9800,430,660,4951,188,4189.0,10750.0,14939,329812,189320.0
9,2020-11-08T17:00:00,11,Marche,493,69,562,9679,10241,414,502,7498,1050,18600.0,189.0,18789,345496,202915.0


In [24]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-09T17:00:00,1,Piemonte,4540,312,4852,51404,56256,2059,2876,39721,4678,43124.0,57531.0,100655,1162525,735109.0
1,2020-11-09T17:00:00,2,Valle d'Aosta,159,15,174,2092,2266,94,201,1863,207,3822.0,514.0,4336,45993,27671.0
2,2020-11-09T17:00:00,3,Lombardia,6414,670,7084,124375,131459,-951,4777,115630,18442,203758.0,61773.0,265531,3277767,2014134.0
3,2020-11-09T17:00:00,4,Trentino-Alto Adige,686,65,751,10637,11388,648,840,12648,816,19836.0,5016.0,24852,568496,NaN
4,2020-11-09T17:00:00,5,Veneto,1464,195,1659,51503,53162,2192,2223,25735,2595,26203.0,55289.0,81492,2462529,953936.0
5,2020-11-09T17:00:00,6,Friuli Venezia Giulia,319,46,365,7467,7832,191,320,6760,456,13080.0,1968.0,15048,583139,243252.0
6,2020-11-09T17:00:00,7,Liguria,1387,92,1479,11242,12721,313,538,22310,1937,25834.0,11134.0,36968,489301,249942.0
7,2020-11-09T17:00:00,8,Emilia-Romagna,1939,204,2143,37487,39630,1778,2025,28978,4824,50134.0,23298.0,73432,1740948,922303.0
8,2020-11-09T17:00:00,9,Toscana,1542,234,1776,42704,44480,1033,2244,19116,1594,52685.0,12505.0,65190,1227725,806480.0
9,2020-11-09T17:00:00,10,Umbria,360,64,424,9572,9996,196,314,5060,197,4303.0,10950.0,15253,330852,189991.0


In [25]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-08T17:00:00,1,Piemonte,4367,304,4671,49526,54197,2708,3884,38953,4629,42321.0,55458.0,97779,1149489,726199.0
1,1,2020-11-08T17:00:00,2,Valle d'Aosta,158,14,172,2000,2172,-9,55,1758,205,3634.0,501.0,4135,45323,27399.0
2,2,2020-11-08T17:00:00,3,Lombardia,6225,650,6875,125535,132410,4781,6318,110001,18343,201314.0,59440.0,260754,3256646,2003397.0
3,19,2020-11-08T17:00:00,4,Trentino-Alto Adige,653,59,712,10028,10740,642,963,12466,806,19130.0,4882.0,24012,565246,NaN
4,3,2020-11-08T17:00:00,5,Veneto,1355,185,1540,49430,50970,3313,3362,25725,2574,26082.0,53187.0,79269,2448503,948701.0
5,4,2020-11-08T17:00:00,6,Friuli Venezia Giulia,301,45,346,7295,7641,281,504,6636,451,12787.0,1941.0,14728,580815,242297.0
6,5,2020-11-08T17:00:00,7,Liguria,1334,81,1415,10993,12408,300,886,22106,1916,25326.0,11104.0,36430,486171,248706.0
7,6,2020-11-08T17:00:00,8,Emilia-Romagna,1836,194,2030,35822,37852,2203,2360,28740,4816,48938.0,22470.0,71408,1728477,914731.0
8,7,2020-11-08T17:00:00,9,Toscana,1474,226,1700,41747,43447,1864,2479,17938,1561,50905.0,12041.0,62946,1214035,799345.0
9,8,2020-11-08T17:00:00,10,Umbria,353,62,415,9385,9800,430,660,4951,188,4189.0,10750.0,14939,329812,189320.0


In [26]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-09T17:00:00,1,Piemonte,4540,312,4852,51404,56256,2059,2876,39721,4678,43124.0,57531.0,100655,1162525,735109.0,4341375
1,2020-11-09T17:00:00,2,Valle d'Aosta,159,15,174,2092,2266,94,201,1863,207,3822.0,514.0,4336,45993,27671.0,125501
2,2020-11-09T17:00:00,3,Lombardia,6414,670,7084,124375,131459,-951,4777,115630,18442,203758.0,61773.0,265531,3277767,2014134.0,10103969
3,2020-11-09T17:00:00,4,Trentino-Alto Adige,686,65,751,10637,11388,648,840,12648,816,19836.0,5016.0,24852,568496,NaN,1074819
4,2020-11-09T17:00:00,5,Veneto,1464,195,1659,51503,53162,2192,2223,25735,2595,26203.0,55289.0,81492,2462529,953936.0,4907704
5,2020-11-09T17:00:00,6,Friuli Venezia Giulia,319,46,365,7467,7832,191,320,6760,456,13080.0,1968.0,15048,583139,243252.0,1211357
6,2020-11-09T17:00:00,7,Liguria,1387,92,1479,11242,12721,313,538,22310,1937,25834.0,11134.0,36968,489301,249942.0,1543127
7,2020-11-09T17:00:00,8,Emilia-Romagna,1939,204,2143,37487,39630,1778,2025,28978,4824,50134.0,23298.0,73432,1740948,922303.0,4467118
8,2020-11-09T17:00:00,9,Toscana,1542,234,1776,42704,44480,1033,2244,19116,1594,52685.0,12505.0,65190,1227725,806480.0,3722729
9,2020-11-09T17:00:00,10,Umbria,360,64,424,9572,9996,196,314,5060,197,4303.0,10950.0,15253,330852,189991.0,880285


In [27]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-09T17:00:00,1,Piemonte,4540,312,4852,51404,56256,2059,2876,39721,4678,43124.0,57531.0,100655,1162525,735109.0,4341375,367
1,2020-11-09T17:00:00,2,Valle d'Aosta,159,15,174,2092,2266,94,201,1863,207,3822.0,514.0,4336,45993,27671.0,125501,20
2,2020-11-09T17:00:00,3,Lombardia,6414,670,7084,124375,131459,-951,4777,115630,18442,203758.0,61773.0,265531,3277767,2014134.0,10103969,983
3,2020-11-09T17:00:00,4,Trentino-Alto Adige,686,65,751,10637,11388,648,840,12648,816,19836.0,5016.0,24852,568496,NaN,1074819,106
4,2020-11-09T17:00:00,5,Veneto,1464,195,1659,51503,53162,2192,2223,25735,2595,26203.0,55289.0,81492,2462529,953936.0,4907704,825
5,2020-11-09T17:00:00,6,Friuli Venezia Giulia,319,46,365,7467,7832,191,320,6760,456,13080.0,1968.0,15048,583139,243252.0,1211357,175
6,2020-11-09T17:00:00,7,Liguria,1387,92,1479,11242,12721,313,538,22310,1937,25834.0,11134.0,36968,489301,249942.0,1543127,209
7,2020-11-09T17:00:00,8,Emilia-Romagna,1939,204,2143,37487,39630,1778,2025,28978,4824,50134.0,23298.0,73432,1740948,922303.0,4467118,516
8,2020-11-09T17:00:00,9,Toscana,1542,234,1776,42704,44480,1033,2244,19116,1594,52685.0,12505.0,65190,1227725,806480.0,3722729,415
9,2020-11-09T17:00:00,10,Umbria,360,64,424,9572,9996,196,314,5060,197,4303.0,10950.0,15253,330852,189991.0,880285,70


In [28]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [29]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-09T17:00:00,3,Lombardia,6414,670,7084,124375,131459,-951,4777,...,68.16,-1160,-1541,21121,10737.0,99,5629,22.62,0.04728,2.63
1,2020-11-09T17:00:00,15,Campania,1949,191,2140,70180,72320,2668,3120,...,44.73,2531,-1481,15793,10438.0,18,434,19.76,0.05392,1.56
2,2020-11-09T17:00:00,1,Piemonte,4540,312,4852,51404,56256,2059,2876,...,85.01,1878,-1008,13036,8910.0,49,768,22.06,0.06625,2.32
3,2020-11-09T17:00:00,9,Toscana,1542,234,1776,42704,44480,1033,2244,...,56.39,957,-235,13690,7135.0,33,1178,16.39,0.06028,1.75
4,2020-11-09T17:00:00,5,Veneto,1464,195,1659,51503,53162,2192,2223,...,23.64,2073,-1139,14026,5235.0,21,10,15.85,0.04530,1.66
5,2020-11-09T17:00:00,12,Lazio,2761,237,2998,50276,53274,1896,2153,...,31.73,1821,-336,22518,15518.0,16,241,9.56,0.03671,1.16
6,2020-11-09T17:00:00,8,Emilia-Romagna,1939,204,2143,37487,39630,1778,2025,...,39.53,1665,-335,12471,7572.0,8,238,16.24,0.04533,1.64
7,2020-11-09T17:00:00,19,Sicilia,1303,187,1490,20449,21939,472,1023,...,34.76,409,-60,8458,4895.0,27,524,12.10,0.02059,0.65
8,2020-11-09T17:00:00,4,Trentino-Alto Adige,686,65,751,10637,11388,648,840,...,61.32,609,-123,3250,NaN,10,182,25.85,0.07815,2.31
9,2020-11-09T17:00:00,16,Puglia,968,119,1087,16726,17813,626,730,...,32.51,542,-36,4103,3929.0,7,97,17.79,0.01821,0.66


In [30]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [31]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,64,6,70,91.43
1,Piemonte,312,55,367,85.01
2,Valle d'Aosta,15,5,20,75.00
3,Lombardia,670,313,983,68.16
4,Trentino-Alto Adige,65,41,106,61.32
5,Toscana,234,181,415,56.39
6,Marche,71,56,127,55.91
7,Campania,191,236,427,44.73
8,Liguria,92,117,209,44.02
9,Emilia-Romagna,204,312,516,39.53


In [32]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])) * 100).round(3)
    return df

In [33]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ./data/csv/regioni/covid19-piemonte.csv
Aggiornato ./data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ./data/csv/regioni/covid19-lombardia.csv
Aggiornato ./data/csv/regioni/covid19-veneto.csv
Aggiornato ./data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ./data/csv/regioni/covid19-liguria.csv
Aggiornato ./data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ./data/csv/regioni/covid19-toscana.csv
Aggiornato ./data/csv/regioni/covid19-umbria.csv
Aggiornato ./data/csv/regioni/covid19-marche.csv
Aggiornato ./data/csv/regioni/covid19-lazio.csv
Aggiornato ./data/csv/regioni/covid19-abruzzo.csv
Aggiornato ./data/csv/regioni/covid19-molise.csv
Aggiornato ./data/csv/regioni/covid19-campania.csv
Aggiornato ./data/csv/regioni/covid19-puglia.csv
Aggiornato ./data/csv/regioni/covid19-basilicata.csv
Aggiornato ./data/csv/regioni/covid19-calabria.csv
Aggiornato ./data/csv/regioni/covid19-sicilia.csv
Aggiornato ./data/csv/regioni/covid19-sardegna.csv
